# a
[【中文分词系列】 3. 字标注法与HMM模型 - 科学空间|Scientific Spaces](https://kexue.fm/archives/3922#Python%E5%AE%9E%E7%8E%B0)

[一次性弄懂马尔可夫模型、隐马尔可夫模型、马尔可夫网络和条件随机场！(词性标注代码实现) - 知乎](https://zhuanlan.zhihu.com/p/74063873) 没啥用

~~数据参考 https://github.com/pwxcoo/chinese-xinhua~~

TODO
1. 数据参考应该改成jieba的
2. 重新写一个，并对比苏神的

存在问题：
1. 最后推出来的公式少了$P(O_1)$

暂停：
已经搞懂维提比和HMM，先看CRF

In [2]:
FILE_PATH = '../data/chinese-xinhua-master/data/ci.json'

In [4]:
import pandas as pd
import json
df = None
with open(FILE_PATH, encoding='utf-8') as f:
    
    word = json.load(f)
    l = [len(w['ci']) for w in word]
    df = pd.DataFrame(l)
df.describe()

,0
count,264434.000000
mean,2.375156
std,1.020124
min,1.000000
25%,2.000000
50%,2.000000
75%,2.000000
max,15.000000


In [5]:
from collections import Counter
from math import log
import json
hmm_model = {i:Counter() for i in 'sbme'}
with open(FILE_PATH, encoding='utf-8') as f:
    
    word = json.load(f)
    cnt = 1 # 没有词频，假设每个词都出现一次，待改进
    for w in word:
        w = w['ci']
        
        if len(w) == 1:
            hmm_model['s'][w[0]] += cnt
        else:
            hmm_model['b'][w[0]] += cnt
            hmm_model['e'][w[-1]] += cnt

            for m in w[1:-1]:hmm_model['m'][m] += cnt
#         break;

hmm_model字典里面计算的是 $P(\lambda_k|o_k)$ 的分子部分

下面计算分母部分

In [6]:
log_total = {i:log(sum(v.values())) for i,v in hmm_model.items()}; log_total # 

{'s': 5.805134968916488,
 'b': 12.484090671145607,
 'm': 11.5082746667548,
 'e': 12.484090671145607}

猜出来的$P(o_k|o_{k-1})$,即知道$o_{k-1}$，去枚举$o_k$，共bmes三种标签，因此有4*4 = 9项

枚举$o_{k-1}$再枚举$o_{k}$，$\forall o_{k-1} \in bems, \sum_{o_{k} \in bems }{P(o_{k}|o_{k-1})} = 1, $
1. b
    1. ~~b~~
    1. m 1
    1. e 1
    2. ~~s~~
2. m
    1. ~~b~~
    2. m 1
    3. e 1
    4. ~~s~~
3. e
    1. b 1
    2. ~~m~~
    3. ~~e~~
    4. s 1
4. s
    1. b 1
    2. ~~m~~
    3. ~~e~~
    4. s 1

In [7]:
trans = {'ss':0.3,
    'sb':0.7,
    'bm':0.3,
    'be':0.7, 
    'mm':0.3,
    'me':0.7,
    'es':0.3,
    'eb':0.7
 }

# nodes

是一个$ \{ P(\lambda_k|o_k) \}$矩阵  

列： $o_k \in bems$ 

行：$\lambda_k \in input\_string$

# paths
```json
{'s': -5.805134968916488, 'b': -8.900571732689496, 'm': -7.770605048471433, 'e': -9.188253805141278}
{'ss': -11.310269937832976, 'sb': -13.97830772741787, 'mm': -14.251491896513892, 'me': -14.939575202775782}
{'sss': -16.122257726189517, 'ssb': -19.56800008436242, 'sbm': -23.57714448173857, 'sbe': -23.816488249508165}
```


```json
// <bmes路径>[-1] 应该互不相同
// <bmes路径>[-2:] in trans 才有效
{
<bmes路径>:叠加概率
}
```

In [11]:
# https://stackoverflow.com/questions/49051492/index-generates-attributeerror-dict-values-object-has-no-attribute-index
# python2 to python3
import operator
def get_key_by_max_value(d):
    """
    https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary
    """
    return max(d.items(), key=operator.itemgetter(1))[0]

def viterbi(nodes):
    paths = nodes[0]
    for node in nodes[1:]: 
        # l 每个字
        
        paths_ = paths
        print(paths)
        paths = {}
        for i,p_o in node.items():
            # o_k 这个字的 bems
            nows = {}
            for path, path_p in paths_.items(): # 到 l - 1 为止 最大 的 <=4 维特比 路径 ；每条路径长度(paths的key) 是 j 即 nodes[0:j] 计算得到
                
                trans_key = path[-1] + i
                current_path = path + i
                
                if trans_key in trans: # o_{i-1} + o{i}
                    nows[ current_path ]= path_p + p_o + trans[trans_key]
                    
            k = get_key_by_max_value(nows) 
            
            assert i == k[-1]
            
            paths[k] = nows[k]
    return get_key_by_max_value(paths)

def hmm_cut(s):
    nodes = [{i:log(j[t]+1)-log_total[i] for i,j in hmm_model.items()} for t in s]
    tags = viterbi(nodes)
    words = [s[0]]
    for i in range(1, len(s)):
        if tags[i] in ['b', 's']:
            words.append(s[i])
        else:
            words[-1] += s[i]
    return words
(hmm_cut('我是傻逼'))

{'s': -5.805134968916488, 'b': -8.900571732689496, 'm': -7.770605048471433, 'e': -9.188253805141278}
{'ss': -11.310269937832976, 'sb': -13.97830772741787, 'mm': -14.251491896513892, 'me': -14.939575202775782}
{'sss': -16.122257726189517, 'ssb': -19.56800008436242, 'sbm': -23.57714448173857, 'sbe': -23.816488249508165}


['我', '是', '傻', '逼']

4

In [51]:
df

""
b
e
m
s


In [ ]:
from collections import Counter
from math import log

hmm_model = {i:Counter() for i in 'sbme'}

with open('dict.txt') as f:
    for line in f:
    lines = line.decode('utf-8').split(' ')
    if len(lines[0]) == 1:
    hmm_model['s'][lines[0]] += int(lines[1])# 应该是频次吧
    else:
    hmm_model['b'][lines[0][0]] += int(lines[1])
    hmm_model['e'][lines[0][-1]] += int(lines[1])
    for m in lines[0][1:-1]:
    hmm_model['m'][m] += int(lines[1])

log_total = {i:log(sum(hmm_model[i].values())) for i in 'sbme'}

trans = {'ss':0.3,
    'sb':0.7,
    'bm':0.3,
    'be':0.7, 
    'mm':0.3,
    'me':0.7,
    'es':0.3,
    'eb':0.7
 }

trans = {i:log(j) for i,j in trans.iteritems()}

def viterbi(nodes):
    paths = nodes[0]
    for l in range(1, len(nodes)):
        paths_ = paths
        paths = {}
        for i in nodes[l]:
            nows = {}
            for j in paths_:
            if j[-1]+i in trans:
                nows[j+i]= paths_[j]+nodes[l][i]+trans[j[-1]+i]
            k = nows.values().index(max(nows.values()))
            paths[nows.keys()[k]] = nows.values()[k]
    return paths.keys()[paths.values().index(max(paths.values()))]

def hmm_cut(s):
    nodes = [{i:log(j[t]+1)-log_total[i] for i,j in hmm_model.iteritems()} for t in s]
    tags = viterbi(nodes)
    words = [s[0]]
    for i in range(1, len(s)):
        if tags[i] in ['b', 's']:
            words.append(s[i])
        else:
            words[-1] += s[i]
    return words